# CNN Architectures applied to Caltech256

This notebook expands on the concepts in chapter 14 by looking at differen architectures for CNNs

Watch the [Chapter 14 Walkthough Video by Tom](https://youtu.be/iDiE17lk0X4)


In this notebook we will apply different CNNs to the [Caltech256 image database](http://www.vision.caltech.edu/Image_Datasets/Caltech256/). There are 30,607 images in this dataset spanning 257 object categories. Object categories are extremely diverse, ranging from grasshopper to tuning fork. The images sizes range from about 150x150 pixels to 600x450 pixels



# Section 0

=== *You must run this section to set up things for any of the sections below * ===
### Setting up Python tools



We'll use three libraries for this tutorial: 
- [pandas](http://pandas.pydata.org/) : dataframes for spreadsheet-like data analysis, reading CSV files, time series
- [numpy](http://www.numpy.org/) : for multidimensional data and linear algebra tools
- [matplotlib](http://matplotlib.org/) : Simple plotting and graphing
- [seaborn](http://stanford.edu/~mwaskom/software/seaborn/) : more advanced graphing
-  [scikit-learn](https://scikit-learn.org/stable/) : provides many machine learning algorithms and tools to training and test.


In [6]:
# First, we'll import pandas and numpy, two data processing libraries
import pandas as pd
import numpy as np

# We'll also import seaborn and matplot, twp Python graphing libraries
import seaborn as sns
import matplotlib.pyplot as plt
# Import the needed sklearn libraries
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

# The Keras library provides support for neural networks and deep learning
# Use the updated Keras library from Tensorflow -- provides support for neural networks and deep learning
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Lambda, Flatten, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from keras.utils import np_utils

# We will turn off some warns in this notebook to make it easier to read for new students
import warnings
warnings.filterwarnings('ignore')

In [2]:
# If using colab, upload the two zip files 
# This will unzip this files into folders 
!unzip /content/test_resized.zip
!unzip /content/train_resized.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


# Section 1: Set up the CNN



In [2]:
# Set up the Neural Network
input_shape=(224,224,3)
output_Size = 259

NN = Sequential()
NN.add(Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))
NN.add(Conv2D(8, (3, 3), activation='relu'))
NN.add(MaxPooling2D(pool_size=(2, 2)))
NN.add(Conv2D(16, (3, 3), activation='relu'))
NN.add(Conv2D(16, (3, 3), activation='relu'))
NN.add(MaxPooling2D(pool_size=(2, 2)))
NN.add(Conv2D(32, (3, 3), activation='relu'))
NN.add(MaxPooling2D(pool_size=(2, 2)))
NN.add(Conv2D(64, (3, 3), activation='relu'))
NN.add(MaxPooling2D(pool_size=(2, 2)))
NN.add(Flatten())
NN.add(Dense(256, activation='relu'))
NN.add(Dense(output_Size, activation='softmax'))
print ("Neural Network Model created")
NN.summary()

# Compile neural network model
NN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Neural Network Model created
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 8)       224       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 8)       584       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 16)      1168      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)          

In [3]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=5, 
                                            verbose=2, 
                                            factor=0.5,                                            
                                            min_lr=0.000001)

early_stops = EarlyStopping(monitor='loss', 
                            min_delta=0, 
                            patience=20, 
                            verbose=2, 
                            mode='auto')

checkpointer = ModelCheckpoint(filepath = 'cis3115_MNIST.{epoch:02d}-{accuracy:.6f}.hdf5',
                               verbose=2,
                               save_best_only=True, 
                               save_weights_only = True)


Using TensorFlow backend.


In [4]:
# Read data from the folders

#train_folder = 'train_resized'
#validate_folder = 'test_resized'
train_folder = 'train'
validate_folder = 'test'

print ("Reading training images from: " ,train_folder)
print ("Reading validation images from: " ,validate_folder)


Reading training images from:  train
Reading validation images from:  test


In [7]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
#        rotation_range=10,
#        width_shift_range=0.1,
#        height_shift_range=0.1,
        rescale=1./255,
#        shear_range=0.1,
#        zoom_range=0.1,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
batch_size = 8
#batch_size = 32              # This might take too much memory

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_folder,  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        color_mode="rgb", 
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validate_folder,
        target_size=(224, 224),
        batch_size=batch_size,
        color_mode="rgb", 
        class_mode='categorical')

Found 24536 images belonging to 259 classes.
Found 6071 images belonging to 259 classes.


## Train the Neural Network


In [12]:
# check Pillow version number
from PIL import Image
print('Pillow Version:', PIL.__version__)


ModuleNotFoundError: No module named 'PIL'

In [10]:
# Train the model with the images in the folders
#history = NN.fit_generator(
history = NN.fit(
        train_generator,
        steps_per_epoch=1000,                   # Number of images to process per epoch 
        epochs=100,                              # Number of epochs
        callbacks=[learning_rate_reduction, early_stops],
        validation_data=validation_generator,
        validation_steps=100 )                  # Number of images from validation set to test
#NN.save_weights('cifar_weights.h5')  # always save your weights after training or during training


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

## Plot the Training History

We store the performance during training is a variable named 'history'. The x-axis is the training time or number of epochs.

- Accuracy: Accuracy of the predictions, hopefully this is increasing to near 1.0
- Loss: How close the output is to the desired output, this should decrease to near 0.0

In [ ]:
# 10. Evaluate model on test data
print ("Running final scoring on test data")
score = NN.evaluate(X_test, y_test, verbose=1)
print ("The accuracy for this model is ", format(score[1], ",.2f"))

# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)

ax[0].plot(history.history['acc'], color='b', label="Training accuracy")
ax[0].plot(history.history['val_acc'], color='r',label="Testing accuracy")
ax[0].set_title("Accruacy")
legend = ax[0].legend(loc='best', shadow=True)
              
ax[1].plot(history.history['loss'], color='b', label="Training loss")
ax[1].plot(history.history['val_loss'], color='r', label="Testing loss",axes =ax[1])
ax[1].set_title("Loss")
legend = ax[1].legend(loc='best', shadow=True)
plt.ylim(0,1)